# Using constraints in optimization loop

Sigurd Carlsen, June 2019.

In real world experiments it is not always possible to maintain the same space as was specified when the experiment was created. For example one could run out of a certain material for the experiment.

In these cases constraints come in handy. Instead of redefining our space we just implement constraints on the parameter values that can be used in the experiment. These constraints can be removed or changed as the exeriment changes.

So far we have 5 types of constraints: Single, Inclusive, Exclusive, Sum and Conditional.

- "Single" constraints enforces a specific dimension to take on a specific value.
Example: $x = 2.0$
- "Inclusive" constraints forces values to only be the ones included in the constraint.
Example: $x ∈ [2.0, 3.0]$
- "Exclusive" constraints forces values to only be the ones excluded in the constraint.
Example: $x ∉ [2.0, 3.0]$
- "Sum" constraints forces the sum of values of a specific dimension to either be greater than or equal to or less than or equal to a certain value.
For example: $x0 + x1 + x2 < 4$
- "Conditional" constraints is a set of constraints that should only be applied under certain conditions.
For example: if $x0 > 2$ then $x1 = 10$ else $x1 = 0$

This notebook is an example on how to use these constraints.
Note that constraints can only be used in the optimizer in conjunction with ask() and tell()

Let's start by importing our packages:

In [1]:
from ProcessOptimizer.learning import GaussianProcessRegressor
from ProcessOptimizer import Optimizer
from ProcessOptimizer import Integer, Categorical, Real, Space
import numpy as np
import time
np.random.seed(1234)

In [2]:
from ProcessOptimizer.space.constraints import Single, Inclusive, Exclusive, Sum, Conditional
from ProcessOptimizer.space.constraints import Constraints

Notes about the constraint classes. The `Single`, `Inclusive`, `Exclusive`, `Sum` and `Conditional` classes are all specific types of constraints, in the same way that `Integer`, `Categorical` and `Real` are specific types of dimensions.

The `Constraints` class is a class that holds all the constraints in the same way that the `Space` class holds all the dimensions.

Let us define our space:

In [3]:
space = [
        Real(1, 4),
        Integer(1, 4),
        Categorical(list('abcd')),
    ]

We initialize our optimizer and asks for 5 samples.

In [4]:
opt = Optimizer(space, "GP")
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.2343118064815393, 4, 'a'], [3.5060862359092537, 3, 'd'], [2.1765381816450544, 3, 'd'], [1.6200909740185605, 2, 'a'], [2.8338292820563438, 3, 'b']]


After the optimizer has been initialized we can add constraints to it.
This is done by first creating a list of all constraints we want to pass on to the optimizer.
Let us start out simple by creating a list with only one constraint.
First we try the "Single" type constraint.

In [5]:
constraints_list = [
    Single(0, 2.0, 'real')
]

The first argument to "Single" is the index of the dimension for which the constraint should be applied.
The second argument is the value of the constraint, and the third argument is the type of dimension.

Here we have created a constraint on dimension 0. The type of dimension is set to 'real' as the type of dimension 0 in our space object is of type Real. Because the dimension is of type 'real' the value should be a float which is why it is set to "2.0" and not "2"

Now we add the constraints to the optimizer and asks for 5 samples to evaluate.
Note that adding constraints should always be done by calling set_constraints() to ensure that the optimizer internally handles new constraints correctly. 

In [6]:
opt.set_constraints(constraints_list)

We can get the constraints by calling opt.get_constraints()

In [7]:
opt.get_constraints()

Constraints([Single(dimension=0, value=2.0, dimension_type=real)])

In [8]:
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.0, 3, 'c'], [2.0, 3, 'd'], [2.0, 1, 'a'], [2.0, 3, 'a'], [2.0, 1, 'a']]


Note how all of the samples satisfies the constraint: Dimension 0 is alawys 2.0

We can remove the constraints again simply by calling remove_constraints().

In [9]:
opt.remove_constraints()
next_x = opt.ask(n_points = 5)
print(next_x)

[[3.6039085869467122, 2, 'b'], [2.1367445745380658, 1, 'a'], [2.8681693339273946, 1, 'd'], [2.362511314366296, 2, 'c'], [2.028521409840995, 4, 'a']]


Now let us try with two different Inclusive constraints

In [10]:
constraints_list = [
    Inclusive(1, (2, 3), 'integer'),
    Inclusive(2, ('b', 'c'), 'categorical')  
]

Note how we change the dimension_type arguments to 'integer' and 'categorical' to correspond to the "Real" and "Categorical" dimension types of dimension 1 and 2.
The second argument of Inclusive constraints should be a tuple of length 2 for 'integer' and 'real' dimensions. This tuple defines the lower and upper bound of the constraint.
For categorical dimensions the constraint is handled a little bit differently, as upper and lower bounds does not make sense in categorical dimensions. This means that for categorical dimensions the second argument can be a tuple with a lenght of more than than 2, and that any values in this tuple, will be the only values that can be sampled for the given dimension.

In [11]:
opt.set_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.8721856597656277, 2, 'c'], [2.181347056881595, 2, 'c'], [3.371866307783935, 2, 'b'], [2.6443587177454155, 2, 'b'], [1.9837362525955056, 3, 'c']]


Now only values between and including 2 and 3 are drawn for dimension 1 and only categorical values of 'b' or 'c' are drawn for dimension 2.
Note that calling set_constraints always overwrites all existing constraints in the optimizer

Exlusive constraints are called the same way as Inclusive constraints but instead of enforcing that values drawn lie inside the bounds of the constraints, the values are instead forced to be outside of the bounds of the constraints.

In [12]:
constraints_list = [
    Exclusive(1, (2, 3), 'integer'),
    Exclusive(2, ('b', 'c'), 'categorical')  
]

In [13]:
opt.set_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[3.946014224565864, 1, 'a'], [2.673282485282349, 4, 'a'], [2.373434946692283, 4, 'd'], [2.9726553314734585, 1, 'a'], [2.704874213977322, 1, 'd']]


Note how values of 'b' and 'c' are now illegal.

We can use multiple constraints on the same dimension as long as they do not conflict with eachother. For example it is not possible to put two different "Single" constraints on the same dimension.

Here we want values between 1.5 and 2.0 and values between 3.0 and 3.5 to be excluded:

In [14]:
constraints_list = [
    Exclusive(0, (1.5, 2.0), 'real'),
    Exclusive(0,( 3.0, 3.5), 'real')
]
opt.set_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.6976957818171687, 3, 'c'], [2.1056730463004563, 4, 'a'], [3.549790083014442, 4, 'b'], [2.0174059046497073, 2, 'b'], [2.9563165016091553, 4, 'c']]


The same can be done with Inclusive constraints

In [15]:
constraints_list = [
    Inclusive(0, (1.0, 1.1), 'real'),
    Inclusive(0, (2.0, 2.1), 'real'),
    Inclusive(0, (3.0, 3.1), 'real')
]
opt.set_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.0968572621015573, 3, 'd'], [1.093104583399998, 2, 'c'], [2.0049879066757823, 1, 'd'], [1.037220945183141, 4, 'd'], [2.0552404647627665, 3, 'b']]


We can even mix Inclusive and Exclusive constraints for the same dimension

In [16]:
constraints_list = [
    Inclusive(0, (2.0, 3.0), 'real'),
    Exclusive(0, (2.3, 2.7), 'real')
]
opt.set_constraints(constraints_list)
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.2815152018868297, 3, 'b'], [2.1624746189422055, 2, 'c'], [2.7240287944217485, 2, 'b'], [2.8130063240803054, 2, 'd'], [2.8202279743247756, 2, 'b']]


Whatch out for creating invalid constraint combinations. For example: `Inclusive(0,(2.0,3.0),'real')` and `Exclusive(0,(1.5,3.5),'real')` leaves no valid values to be sampled.

## Using the rvs method
So far we have been using the `opt.ask()` method for drawing samples. We do this to showcase that the constraints are part of the optimizer. However the `opt.ask()` method is inherently very slow when asking for more than a handfull of samples. Therefore if we want to test the constraints we should use the Constraints class instead. This is the class that are being called internally when adding constraints to the optimizer.
The initialitation of a Constraints object requires two arguments: constraints_list and space.
The space argument can either be a list of dimensions or a Space object:

In [17]:
# This works
constraints = Constraints(constraints_list,space)

In [18]:
# This also works
space_object = Space(space)
constraints = Constraints(constraints_list,space_object)

In [19]:
constraints.rvs(n_samples = 10)

[[2.233698630357855, 1, 'd'],
 [2.911484671439348, 3, 'c'],
 [2.876265731487022, 4, 'a'],
 [2.094607904691135, 3, 'b'],
 [2.0527363421746516, 2, 'b'],
 [2.715183765325116, 1, 'c'],
 [2.783006175879051, 4, 'c'],
 [2.197260366051837, 3, 'a'],
 [2.0952784143703926, 2, 'c'],
 [2.8214030333511184, 3, 'a']]

We can also test if a sample that we are interested in lives up to the constraints

In [20]:
sample_valid = [2.1, 2, 'b']
sample_not_valid = [1, 2, 'b']

In [21]:
constraints.validate_sample(sample_valid)

True

In [22]:
constraints.validate_sample(sample_not_valid)

False

## Sum constraints
"Sum" is a type of constraints where the sum of specified dimensions should be either less or more than a certain value. Sum constraints only work for non-categorical dimensions.
For example if we want to write a constraint like: x0 + x1 < 3:

In [23]:
cons = Sum((0,1),3)

Here the first argument which is a tuple defines the dimensions which should be included in the sum. In this case dimension 0 and 1. The second argument defines the max value of this sum. In this case 3.

In [24]:
opt.set_constraints([cons])
next_x = opt.ask(n_points = 5)
print(next_x)

[[1.1343067474153847, 1, 'b'], [1.421711938375049, 1, 'a'], [1.70963763977009, 1, 'b'], [1.3825687019633712, 1, 'c'], [1.9204913411056788, 1, 'b']]


If we want the sum to always be larger than a certain value we pass an optional parameter "less_than". If less_than is True the sums should be less than or equal to the value given in constraint. If less_than equals False the sum should be larger than or equal the given parameter:

In [25]:
constraint = Sum((0,1),4, less_than = False)
opt.set_constraints([constraint])
next_x = opt.ask(n_points = 5)
print(next_x)

[[2.9097449117167056, 3, 'b'], [2.9154486050690873, 3, 'd'], [3.3709296714906563, 3, 'a'], [3.4277372366794823, 2, 'c'], [1.3831158856909544, 4, 'a']]


## Conditional constraints
"Conditional" constraints is a way of applying constraints under certain conditions. The "Conditional" class takes 3 arguments where the last two are optional: (condition, if_true, if_false).
The first argument should be a constraint. The second two arguments should be a constraint or a list/tuple of constraints.
The way this works is as follows: If the constraint from "condition" i satisfied then the constraints from "if_true" is applied. If not the constraint from "if_false" is applied instead. if_true and if_false are both optional arguments. If no if_true constraint is passed to the function and the condition is satisfied no constraint will be applied. The same goes for the if_false constraint.

Lets say we want to make the following conditional constraint: If x0 is less than or equal to 2.5 then x2 should be 'a'. Else, x2 should be 'b'.

First we start out be creating the condition. The condition should be a constraint that will evaluate to true if x0 < 2.5 and false otherwise. For this we can use the "Inlcusive" constraint.

In [26]:
condition = Inclusive(0,(1.0,2.5),'real')

If the condition evaluates to true we want x2 to be equal to 'a' and if it evaluates to false we want x2 to be equal to 'b'. This corresponds to two "Single" constraints:

In [27]:
if_true = Single(2,('a'),'categorical')
if_false = Single(2,('b'),'categorical')

Now we put it all together:

In [28]:
constraint_0 = Conditional(condition,if_true = if_true, if_false = if_false)
opt.set_constraints([constraint_0])
next_x = opt.ask(n_points = 5)
print(next_x)

[[3.5430877873925217, 3, 'b'], [1.705129678427533, 3, 'a'], [2.1401543624099753, 2, 'a'], [2.5550651664243604, 2, 'b'], [3.645316683325362, 3, 'b']]


We can also pass a tuple of constraints to the if_true or if_false arguments:

In [29]:
if_true = (Single(2,('a'),'categorical'),Single(1,2,'integer'))
constraint_1 = Conditional(condition,if_true = if_true, if_false = if_false)
opt.set_constraints([constraint_1])
next_x = opt.ask(n_points = 5)
print(next_x)

[[3.658850241695485, 3, 'b'], [3.5877243011879614, 2, 'b'], [3.5167327343478427, 2, 'b'], [3.627977389519116, 3, 'b'], [1.0806490770824209, 2, 'a']]


This was a simple example but because the "if_true" and "if_false" arguments can be conditional constraints as well, we can make very complex constraints by nesting a conditional constraints inside eachother. Say for example that we only want the above conditional constraint to be applied if x1 is larger than 2. This can be achieved as follows:

In [30]:
constraint_2 = Conditional(
    Exclusive(1,(1,2),'integer'),
    if_true =constraint_0,
)

Note how it is not nessecary to give both a "if_true" and "if_false" argument. For example in this case no constraint will be applied if the Exclusive constraint is not satisfied.

In [31]:
constraint_2.validate_sample([1,3,'a'])

True